In [ ]:
# Key Concepts
'''
1. Forward problem: θ → x (simulation)
2. Inverse problem: x → θ (what we're solving)
3. Posterior: P(θ|x) probability distribution over parameters
4. Prior: Initial assumptions about parameter ranges
'''
import pandas as pd
import os
import numpy as np
import sys

# seaparate into train and test set.
from sklearn.model_selection import train_test_split

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
import torch
from torch.distributions import Uniform, ExpTransform, TransformedDistribution #, AffineTransform
import torch.nn as nn
from sklearn.preprocessing import Normalizer
import joblib

import ili
from ili.dataloaders import NumpyLoader
from ili.inference import InferenceRunner
from ili.validation.metrics import PosteriorCoverage, PlotSinglePosterior

from sbi.utils.user_input_checks import process_prior

sys.path.append("/disk/xray15/aem2/camels/proj2")
from setup_params_alice import *
from priors_SB import initialise_priors_SB28

# parameters
device = "cuda" if torch.cuda.is_available() else "cpu"
model = "IllustrisTNG"
spec_type = "attenuated"
sps = "BC03"
snap = ["044"]
# 12 bins!
n_bins_lf = 13
n_bins_colour = 13
bands = "all" # or just GALEX?
colours = False  # just for now, lets do UVLF
luminosity_functions = True
name = f"{model}_{bands}_{sps}_{spec_type}_{n_bins_lf}"#_{n_bins_colour}"

# initialize CAMELS and load parameter info using camels.py
cam = camels(model=model, sim_set='SB28')

# trys to use Chris method here, get_x, get_theta directly from photometry files instead of intermediate txt files.


In [ ]:
# parameter info file (df_info) is used for defining priors
# the actual parameter values come from the camels class which reads CosmoAstroSeed_IllustrisTNG_L25n256_SB28.txt

#  parameters defined here: /disk/xray15/aem2/data/28pams/IllustrisTNG/SB/CosmoAstroSeed_IllustrisTNG_L25n256_SB28.txt which is used for theta
df_pars = pd.read_csv('/disk/xray15/aem2/data/28pams/IllustrisTNG/SB/CosmoAstroSeed_IllustrisTNG_L25n256_SB28.txt', delim_whitespace=True)
print(df_pars)


# prior values come from this:
df_info = pd.read_csv("/disk/xray15/aem2/data/28pams/Info_IllustrisTNG_L25n256_28params.txt")
print(df_info)



In [ ]:
theta = df_pars.iloc[:, 1:29].to_numpy()  # excluding 'name' column and 'seed' column

print(theta)
print(theta.shape)

In [ ]:
print("Column names:")
print(df_pars.columns.tolist())

In [ ]:
# plot the first one (omega0) to see shape of prior:
plt.hist(theta[:, 24])

In [ ]:
'''  reference from setup_params_alice.py
def calc_df(_x, volume, massBinLimits):
    hist, _dummy = np.histogram(_x, bins=massBinLimits)
    hist = np.float64(hist)
    phi = (hist / volume) / (massBinLimits[1] - massBinLimits[0])

    phi_sigma = (np.sqrt(hist) / volume) / (
        massBinLimits[1] - massBinLimits[0]
    )  # Poisson errors

    return phi, phi_sigma, hist

def get_luminosity_function(
    photo,
    filt,
    lo_lim,
    hi_lim,
    n_bins=15,
    mask=None,
):
    h = 0.6711
    if mask is None:
        mask = np.ones(len(photo[filt]), dtype=bool)

    binLimits = np.linspace(lo_lim, hi_lim, n_bins)
    phi, phi_sigma, hist = calc_df(photo[filt][mask], (25 / h) ** 3, binLimits)
    phi[phi == 0.0] = 1e-6 + np.random.rand() * 1e-7
    phi = np.log10(phi)
    return phi, phi_sigma, hist, binLimits

def get_photometry(
    sim_name="LH_0",
    spec_type="attenuated",
    snap="090",
    sps="BC03",
    model="IllustrisTNG",
    photo_dir=("/disk/xray15/aem2/data/28pams/IllustrisTNG/photometry"),
    filters=[
        "SLOAN/SDSS.u",
        "SLOAN/SDSS.g",
        "SLOAN/SDSS.r",
        "SLOAN/SDSS.i",
        "SLOAN/SDSS.z",
        "GALEX FUV",
        "GALEX NUV",
    ],
):
    photo_file = f"{photo_dir}/{model}_{sim_name}_photometry.hdf5"
    photo = {}
    with h5py.File(photo_file, "r") as hf:
        for filt in filters:
            photo[filt] = hf[
                f"snap_{snap}/{sps}/photometry/luminosity/{spec_type}/{filt}"
            ][:]
            photo[filt] *= unyt_quantity.from_string("1 erg/s/Hz") 
            photo[filt] = lnu_to_absolute_mag(photo[filt])

    return photo
'''
def get_photometry_SB(
    sim_name="SB28_0",
    spec_type="attenuated",
    snap="044",
    sps="BC03",
    model="IllustrisTNG",
    photo_dir="/disk/xray15/aem2/data/28pams/IllustrisTNG/SB/photometry",
    filters=["GALEX FUV", "GALEX NUV"],
):
    photo = {}
    with h5py.File(f"{photo_dir}/alice_galex.h5", "r") as hf:
        for filt in filters:
            path = f"{sim_name}/snap_{snap}/{sps}/photometry/luminosity/{spec_type}/{filt}"
            photo[filt] = hf[path][:]
            # comment out if data is already in desired format, remove conversion if not needed. in Chris function get_photometry so keep here.
            photo[filt] *= unyt_quantity.from_string("1 erg/s/Hz")
            photo[filt] = lnu_to_absolute_mag(photo[filt])
    return photo

def get_theta_SB(model="IllustrisTNG", device="cuda"):
    # theta is the number of simulation parameters so 28
    cam = camels(model=model, sim_set='SB28')
    theta = np.array([
        cam.params['Omega0'].values,              # Omega0
        cam.params['sigma8'].values,              # sigma8
        cam.params['WindEnergyIn1e51erg'].values, # Wind Energy in 1e51 erg
        cam.params['RadioFeedbackFactor'].values,  # Radio Feedback Factor
        cam.params['VariableWindVelFactor'].values, # Variable Wind Velocity Factor
        cam.params['RadioFeedbackReiorientationFactor'].values, # Radio Feedback Reorientation Factor
        cam.params['OmegaBaryon'].values,         # Omega Baryon
        cam.params['HubbleParam'].values,         # Hubble Parameter
        cam.params['n_s'].values,                 # n_s
        cam.params['MaxSfrTimescale'].values,     # Max SFR Timescale
        cam.params['FactorForSofterEQS'].values,  # Factor for Softer EQS
        cam.params['IMFslope'].values,            # IMF slope
        cam.params['SNII_MinMass_Msun'].values,   # SNII Minimum Mass (Msun)
        cam.params['ThermalWindFraction'].values, # Thermal Wind Fraction
        cam.params['VariableWindSpecMomentum'].values, # Variable Wind Specific Momentum
        cam.params['WindFreeTravelDensFac'].values, # Wind Free Travel Density Factor
        cam.params['MinWindVel'].values,          # Minimum Wind Velocity
        cam.params['WindEnergyReductionFactor'].values, # Wind Energy Reduction Factor
        cam.params['WindEnergyReductionMetallicity'].values, # Wind Energy Reduction Metallicity
        cam.params['WindEnergyReductionExponent'].values, # Wind Energy Reduction Exponent
        cam.params['WindDumpFactor'].values,      # Wind Dump Factor
        cam.params['SeedBlackHoleMass'].values,   # Seed Black Hole Mass
        cam.params['BlackHoleAccretionFactor'].values, # Black Hole Accretion Factor
        cam.params['BlackHoleEddingtonFactor'].values, # Black Hole Eddington Factor
        cam.params['BlackHoleFeedbackFactor'].values, # Black Hole Feedback Factor
        cam.params['BlackHoleRadiativeEfficiency'].values, # Black Hole Radiative Efficiency
        cam.params['QuasarThreshold'].values,     # Quasar Threshold
        cam.params['QuasarThresholdPower'].values # Quasar Threshold Power
    ]).T
    
    return torch.tensor(theta, dtype=torch.float32, device=device)
    

def get_x_SB( # get colours or LFs
    # x.shape= (no. sims, no. bins*features) 

        # 2 GALEX filters (FUV, NUV)
        # Each filter gets n_bins_lf (12) bins
        # 2 filters * 12 bins = 24 features
        # 1 color (FUV-NUV)
        # Gets n_bins_colour (9) bins
        # 1 color * 9 bins = 9 features

    # Total: 24 + 9 = 33 features per simulation with colours & UVLF but with just UVLF its 24
     
    spec_type="attenuated",
    snap="044",
    sps="BC03",
    luminosity_functions=True,
    colours=False, # true for later, lets do UVLF first.
    model="IllustrisTNG",
    photo_dir="/disk/xray15/aem2/data/28pams/IllustrisTNG/SB/photometry",
    n_bins_lf=13,
    n_bins_colour=13,
):
    if isinstance(snap, str):
        snap = [snap]

    x = [[] for _ in range(2048)]  # For SB28 simulations

    for SB28_ in range(2048):
        try:
            for snp in snap:
                photo = get_photometry_SB(
                    sim_name=f"SB28_{SB28_}",
                    spec_type=spec_type,
                    snap=snp,
                    sps=sps,
                    model=model,
                    photo_dir=photo_dir,
                )

                if luminosity_functions:
                    for filt, lo_lim, hi_lim in zip(
                        ["GALEX FUV", "GALEX NUV"],
                        [-20.5, -20.5],
                        [-15, -15],
                    ):
                        phi = get_luminosity_function(
                            photo, filt, lo_lim, hi_lim, n_bins=n_bins_lf
                        )[0]
                        x[SB28_].append(phi)

                if colours:
                    binLimsColour = np.linspace(-0.5, 3.5, n_bins_colour)
                    color = photo["GALEX FUV"] - photo["GALEX NUV"]
                    color_dist = np.histogram(color, binLimsColour, density=True)[0]
                    x[SB28_].append(color_dist)
        except Exception as e:
            print(f"Error processing simulation {SB28_}: {e}")
            x[SB28_] = None

    # Remove any failed simulations
    x = [xi for xi in x if xi is not None]
    
    return x


def get_theta_x_SB(
    spec_type="attenuated",
    snap="044",
    sps="BC03",
    model="IllustrisTNG",
    device="cuda",
    **kwargs,
):
    x = get_x_SB(spec_type=spec_type, snap=snap, sps=sps, model=model, **kwargs)
    theta = get_theta_SB(model=model, device=device)
    
    # Convert x list to proper array format
    x_array = np.array([np.hstack(_x) for _x in x])
    return theta, x_array


if __name__ == "__main__":
    theta, x = get_theta_x_SB()
    print(theta.shape, x.shape)


In [ ]:

# get the priors and data
prior = initialise_priors_SB28(
    df=df_info, 
    device=device,
    astro=True,
    dust=False  # no dust for testing. set to False to only get the 28 model parameters.
    # with dust = True, prior has 32 dimensions (28 parameters + 4 dust parameters) 
)

theta, x = get_theta_x_SB(
    spec_type=spec_type,
    snap=snap,
    sps=sps,
    model=model,
    device=device,
    n_bins_lf=n_bins_lf,
    n_bins_colour=n_bins_colour,
    photo_dir="/disk/xray15/aem2/data/28pams/IllustrisTNG/SB/photometry"
)

# process the data
x_all = np.array([np.hstack(_x) for _x in x])
x_all = torch.tensor(x_all, dtype=torch.float32, device=device)

print("Theta shape:", theta.shape)
print("X shape:", x_all.shape)

In [ ]:
# mask for test fraction (10% currently)
def create_test_mask(n_sims=2048, test_fraction=0.1, random_seed=42):
    """
    Create a test mask similar to the LH one but for SB 2048 simulations.
    
    Args:
        n_sims: Number of simulations (2048 for SB28)
        test_fraction: Fraction of simulations to use for testing (0.1 = 10%)
        random_seed: Random seed for reproducibility
    
    Returns:
        np.array: Boolean mask where True indicates test set
    """
    np.random.seed(random_seed)
    test_mask = np.random.rand(n_sims) > (1 - test_fraction)
    
    # Print some statistics
    print(f"Total simulations: {n_sims}")
    print(f"Test set size: {test_mask.sum()}")
    print(f"Training set size: {(~test_mask).sum()}")
    print(f"Test fraction: {test_mask.sum() / n_sims:.3f}")
    
    # Save the mask
    save_path = "/disk/xray15/aem2/data/28pams/IllustrisTNG/SB/test/test_mask_SB28.txt"
    np.savetxt(save_path, test_mask.astype(int), fmt='%i')
    
    return test_mask


In [ ]:

# Move data to GPU as early as possible
x_all = x_all.to(device)
x_all


In [ ]:
theta = torch.tensor(theta, dtype=torch.float32, device=device)
theta

In [ ]:
# Handle NaN values and normalize while on GPU
x_all_cpu = x_all.cpu().numpy()  # Only move to CPU when necessary for sklearn
x_all_cpu


In [ ]:
print("Data shape before processing:", x_all_cpu.shape)
print("Number of values:",(x_all_cpu).sum())
print("Number of NaN values:", np.isnan(x_all_cpu).sum())
print("Number of infinite values:", np.isinf(x_all_cpu).sum())

# how many nan values are there? if they are all nan something has gone horribly wrong.
# this looks better - 18th Nov

In [ ]:

# get rid of NaN/inf values, replace with small random noise
nan_mask = np.isnan(x_all_cpu) | np.isinf(x_all_cpu)
nan_mask



In [ ]:

if nan_mask.any():
    x_all_cpu[nan_mask] = np.random.rand(np.sum(nan_mask)) * 1e-10


In [ ]:
x_all_cpu

In [ ]:
print("Data shape before processing:", x_all_cpu.shape)
print("Number of NaN values:", np.isnan(x_all_cpu).sum())
print("Number of infinite values:", np.isinf(x_all_cpu).sum())


In [ ]:

# Normalize
norm = Normalizer()
x_all_normalized = norm.fit_transform(x_all_cpu)
x_all = torch.tensor(x_all_normalized, dtype=torch.float32, device=device)
x_all

In [ ]:

# Save normalizer
joblib.dump(norm, f'/disk/xray15/aem2/data/28pams/IllustrisTNG/SB/models/{name}_scaler.save')

# Print final check
print("Any NaN in normalized data:", torch.isnan(x_all).any().item())
print("Any inf in normalized data:", torch.isinf(x_all).any().item())


In [ ]:

# make test mask
test_mask = create_test_mask() # 10% testing
test_mask


In [ ]:
train_mask = ~test_mask # 90% for training
train_mask


# original

hidden_features = 30
num_transforms = 4
nets = [
    # ili.utils.load_nde_sbi(
    #     engine="NLE", model="maf", hidden_features=50, num_transforms=5
    # ),
    ili.utils.load_nde_sbi(
        engine="NPE",
        model="nsf", hidden_features=hidden_features, num_transforms=num_transforms
    ),
    '''

    ili.utils.load_nde_sbi(
        engine="NPE",
        model="nsf", hidden_features=hidden_features, num_transforms=num_transforms
    ),
    ili.utils.load_nde_sbi(
        engine="NPE",
        model="nsf", hidden_features=hidden_features, num_transforms=num_transforms
    ),
    # ili.utils.load_nde_sbi(
    #     engine="NPE",
    #     model="nsf", hidden_features=hidden_features, num_transforms=num_transforms
    # ),
    # ili.utils.load_nde_lampe(model="nsf", device=device, hidden_features=20, num_transforms=2), 
    # ili.utils.load_nde_lampe(model="nsf", device=device, hidden_features=20, num_transforms=2), 
    '''
]
print(nets)


train_args = {"training_batch_size": 4, "learning_rate": 5e-4, 'stop_after_epochs': 20}
print(train_args)


loader = NumpyLoader(
    x=x_all[~test_mask],
    # theta=torch.tensor(theta[~test_mask], device=device)
    theta=torch.tensor(theta[~test_mask, :], device=device)
)
print(loader)

runner = InferenceRunner.load(
    backend="sbi",  #'sbi', # 'lampe',
    engine="NPE",
    prior=prior,
    nets=nets,
    device=device,
    train_args=train_args,
    proposal=None,
    # embedding_net=None,
    out_dir="models/",
    name=name,
)
print(runner)

posterior_ensemble, summaries = runner(loader=loader)
print(posterior_ensemble)

In [ ]:
# Configure network properly
hidden_features = 30
num_transforms = 4
net = ili.utils.load_nde_sbi(
    engine="NPE",                       # Neural Posterior Estimation
    model="nsf",                        # Neural Spline Flow
    hidden_features=hidden_features,    # Network width
    num_transforms=num_transforms,      # Network depth
    # Remove device parameter as it's not allowed
)

# 
'''
Total simulations: 2048
Test set size: 203
Training set size: 1845
Test fraction: 0.099
'''



# Training arguments
train_args = {
    "training_batch_size": 10, # changed from 4 to 10 as dealing with more sims, want it to be faster for initial testing.
    "learning_rate": 1e-3,
    "stop_after_epochs": 20
}

x_train=x_all[train_mask].clone().detach(),
theta_train=theta[train_mask].clone().detach()

# Data loader
loader = NumpyLoader(

    # x = x_all[train_mask]
    # theta=theta[train_mask]
    x_train=x_train,#x_all[train_mask].clone().detach(),
    theta_train=theta_train#theta[train_mask].clone().detach()
)

# Runner setup with device specified here
runner = InferenceRunner.load(
    backend="sbi",
    engine="NPE",
    prior=prior,
    nets=[net],
    device=device,  # Device specified in runner, not network
    train_args=train_args,
    proposal=None,
    out_dir="/disk/xray15/aem2/data/28pams/IllustrisTNG/SB/models/",
    name=name
)

# Run training
posterior_ensemble, summaries = runner(loader=loader)

# process of training:
'''
- the neural network learns P(θ|x): probability of parameters given observations
- uses training data to learn mapping from x → θ
- then we validate on held-out portion of training data
'''


In [ ]:
# plot train/validation loss
fig, ax = plt.subplots(1, 1, figsize=(6,4))
c = list(mcolors.TABLEAU_COLORS)
for i, m in enumerate(summaries):
    ax.plot(m['training_log_probs'], ls='-', label=f"{i}_train", c=c[i])
    ax.plot(m['validation_log_probs'], ls='--', label=f"{i}_val", c=c[i])
ax.set_xlim(0)
ax.set_xlabel('Epoch')
ax.set_ylabel('Log probability')
ax.legend()

In [ ]:
"""
Coverage plots for each model
"""
metric = PosteriorCoverage(
    num_samples=int(4e3),
    sample_method='direct',
    # sample_method="slice_np_vectorized",
    # sample_params={'num_chains': 1},
    # sample_method="vi",
    # sample_params={"dist": "maf", "n_particles": 32, "learning_rate": 1e-2},
    labels=cam.labels,
    plot_list=["coverage", "histogram", "predictions", "tarp"],
    out_dir="/disk/xray15/aem2/plots/28pams/IllustrisTNG/SB/test/sbi_plots",
)
metric

In [ ]:

# 6. Evaluation Metrics
'''
- Coverage: How often true parameters fall within predicted ranges
- TARP: Total Absolute Relative Probability
- Predictions: Compare model predictions with observations
- Parameter recovery: Compare predicted vs true parameters
'''
fig = metric(
    posterior=posterior_ensemble,
    x=x_all[test_mask].cpu(),
    # theta=theta[test_mask].cpu(),
    theta=theta[test_mask, :].cpu(),
    signature=f"coverage_{name}_",
)

In [ ]:
# Now, SBIRunner returns a custom class instance to be able to pass signature strings
# This class has simply for attributes a NeuralPosteriorEstimate and a string list 
print(posterior_ensemble.signatures)

# choose a random input
seed_in = 49
np.random.seed(seed_in)
ind = np.random.randint(len(theta_train))

# generate samples from the posterior using accept/reject sampling
seed_samp = 32
torch.manual_seed(seed_samp)
samples = posterior_ensemble.sample((2048,), torch.Tensor(x_train[ind]).to(device))

# calculate the log_prob for each sample
log_prob = posterior_ensemble.log_prob(samples, torch.Tensor(x_train[ind]).to(device))

samples = samples.cpu().numpy()
log_prob = log_prob.cpu().numpy()

In [ ]:
# Plot the posterior samples and the true value for all pairs of theta values
fig, axs = plt.subplots(5, 5, figsize=(15, 15))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

for i in range(5):
    for j in range(5):
        if i != j:
            axs[i, j].plot(theta_train[ind, i], theta_train[ind, j], 'r+', markersize=10, label='true')
            im = axs[i, j].scatter(samples[:, i], samples[:, j], c=log_prob, s=4, label='samples', cmap='viridis')
            axs[i, j].set_xlabel(f'$\\theta_{i}$')
            axs[i, j].set_ylabel(f'$\\theta_{j}$')
            axs[i, j].legend()
        else:
            axs[i, j].axis('off')

# Add a color bar for log probability
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label='log probability')

plt.show()

print('True values are marked with red pluses, and the pairs of theta values are plotted against each other.')
